In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/deep-past-initiative-machine-translation/sample_submission.csv
/kaggle/input/deep-past-initiative-machine-translation/bibliography.csv
/kaggle/input/deep-past-initiative-machine-translation/publications.csv
/kaggle/input/deep-past-initiative-machine-translation/Sentences_Oare_FirstWord_LinNum.csv
/kaggle/input/deep-past-initiative-machine-translation/OA_Lexicon_eBL.csv
/kaggle/input/deep-past-initiative-machine-translation/eBL_Dictionary.csv
/kaggle/input/deep-past-initiative-machine-translation/train.csv
/kaggle/input/deep-past-initiative-machine-translation/test.csv
/kaggle/input/deep-past-initiative-machine-translation/published_texts.csv
/kaggle/input/deep-past-initiative-machine-translation/resources.csv


In [2]:
train = pd.read_csv('/kaggle/input/deep-past-initiative-machine-translation/train.csv')
test = pd.read_csv('/kaggle/input/deep-past-initiative-machine-translation/test.csv')

In [3]:
test.shape

(4, 5)

In [4]:
for i in range(5):
    print(train['transliteration'].loc[i])
    print('-' * 20)
    print(train['translation'].loc[i])
    print('\n')

KIŠIB ma-nu-ba-lúm-a-šur DUMU ṣí-lá-(d)IM KIŠIB šu-(d)EN.LÍL DUMU ma-nu-ki-a-šur KIŠIB MAN-a-šur DUMU a-ta-a 0.33333 ma-na 2 GÍN KÙ.BABBAR SIG₅ i-ṣé-er PUZUR₄-a-šur DUMU a-ta-a a-lá-ḫu-um i-šu iš-tù ḫa-muš-tim ša ì-lí-dan ITU.KAM ša ke-na-tim li-mu-um e-na-sú-in a-na ITU 14 ḫa-am-ša-tim i-ša-qal šu-ma lá iš-qú-ul 1.5 GÍN.TA a-na 1 ma-na-im i-na ITU.1.KAM ṣí-ib-tám ú-ṣa-áb
--------------------
Seal of Mannum-balum-Aššur son of Ṣilli-Adad, seal of Šu-Illil son of Mannum-kī-Aššur, seal of Puzur-Aššur son of Ataya. Puzur-Aššur son of Ataya owes 22 shekels of good silver to Ali-ahum. Reckoned from the week of Ilī-dan, month of Ša-kēnātim, in the eponymy of Enna-Suen, he will pay in 14 weeks. If he has not paid in time, he will add interest at the rate 1.5 shekel per mina per month.


1 TÚG ša qá-tim i-tur₄-DINGIR il₅-qé
--------------------
Itūr-ilī has received one textile of ordinary quality.


TÚG u-la i-dí-na-ku-um i-tù-ra-ma 9 GÍN KÙ.BABBAR
--------------------
... he did not give you 

In [5]:
!pip install sacrebleu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.7 MB/s eta 0:00:00


In [6]:
import numpy as np
from datasets import Dataset
from transformers import (
    AutoTokenizer, 
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq
)
import sacrebleu
from scipy.stats import gmean
import torch

2026-01-01 18:05:59.929406: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767290760.307190      17 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767290760.419727      17 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767290761.349669      17 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767290761.349744      17 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767290761.349748      17 computation_placer.cc:177] computation placer alr

In [7]:
CONFIG = {
    'model_name': 'google/byt5-small',
    'max_source_length': 256,
    'max_target_length': 256,
    'batch_size': 8,
    'learning_rate': 3e-4,
    'num_epochs': 10,
    'warmup_steps': 500,
    'weight_decay':  0.01,
    'fp16': torch.cuda.is_available(),
    'gradient_accumulation_steps': 4,
}

In [8]:
train

,oare_id,transliteration,translation
0,004a7dbd-57ce-46f8-9691-409be61c676e,KIŠIB ma-nu-ba-lúm-a-šur DUMU ṣí-lá-(d)IM KIŠI...,"Seal of Mannum-balum-Aššur son of Ṣilli-Adad, ..."
1,0064939c-59b9-4448-a63d-34612af0a1b5,1 TÚG ša qá-tim i-tur₄-DINGIR il₅-qé,Itūr-ilī has received one textile of ordinary ...
2,0073f2c0-524c-4bbf-915a-8c1772a4fb98,TÚG u-la i-dí-na-ku-um i-tù-ra-ma 9 GÍN KÙ.BABBAR,... he did not give you a textile. He returned...
3,009fb838-8038-42bc-ad34-5f795b3840ee,KIŠIB šu-(d)EN.LÍL DUMU šu-ku-bi-im KIŠIB ṣí-l...,"Seal of Šu-Illil son of Šu-Kūbum, seal of Ṣilū..."
4,00aa1c55-c80c-4346-a159-73ad43ab0ff7,um-ma šu-ku-tum-ma a-na IŠTAR-lá-ma-sí ù ni-ta...,From Šukkutum to Ištar-lamassī and Nitahšušar:...
...,...,...,...
1556,ff3208e4-8ab8-4368-b4df-7b80afa5bc32,um-ma en-nam-a-šur-ma a-na a-la-ḫi-im qí-bi-ma...,From Ennam-Aššur to Ali-ahum: Here 2 men have ...
1557,ff43a284-3d67-4238-8b4a-9b6cb7531e0a,3 ma-na KÙ.BABBAR ṣa-ru-pá-am i-na ší-im SÍG.Ḫ...,Ilī-ašrannī son of Sukkalliya has received 3 m...
1558,ff5747a4-af8a-4100-a906-a2660ae72606,ša-lim-a-šùr a-na a-mur-IŠTAR ú-ṭá-ḫi-ni-a-tí-...,Šalim-Aššur made us approach Amur-Ištar and Ša...
1559,ff777871-97ce-4bfc-bdfb-73352868944d,a-na en-nam-a-šùr qí-bi-ma um-ma IŠTAR-ra-bi₄-...,To Ennam-Aššur from Ištar-rabiʾat: With respec...


In [9]:
tokenizer = AutoTokenizer.from_pretrained(CONFIG['model_name'])
model = AutoModelForSeq2SeqLM.from_pretrained(CONFIG['model_name'])

tokenizer_config.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [10]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True
)

In [11]:
dataset = Dataset.from_pandas(train)

In [12]:
dataset

Dataset({
    features: ['oare_id', 'transliteration', 'translation'],
    num_rows: 1561
})

In [13]:
def preprocess_function(examples):
    model_inputs = tokenizer(
        examples['transliteration'],
        max_length=CONFIG['max_source_length'],
        truncation=True,
        padding=False
    )

    labels = tokenizer(
        examples['translation'],
        max_length=CONFIG['max_target_length'],
        truncation=True,
        padding=False
    )
    
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

In [14]:
tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset. column_names
)

Map:   0%|          | 0/1561 [00:00<?, ? examples/s]

In [15]:
tokenized_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1561
})

In [16]:
tokenizer.vocab_size

256

In [17]:
tokenizer.pad_token_id

0

In [18]:
len(tokenizer)

384

In [19]:
tokenizer.decode(tokenized_dataset['input_ids'][-1])

'um-ma ša ki-ma a-šùr-i-mì-tí DUMU i-ku-pí-IŠTAR-ma a-šùr-i-mì-tí DUMU e-ta-a a-mur-a-šùr a-šùr-DU₁₀ i-lá-áš-ra-ni-ma a-na ša-lim-a-šùr-ma 1 GÚ AN.NA 10 (TÚG)ku-ta-ni ša a-šùr-i-mì-tí dí-ni-a-tí šu-ma AN.NA ù ku-ta-ni i </s>'

In [20]:
tokenized_dataset['input_ids'][-1]

[120,
 112,
 48,
 112,
 100,
 35,
 200,
 164,
 100,
 35,
 110,
 108,
 48,
 112,
 100,
 35,
 100,
 48,
 200,
 164,
 198,
 188,
 117,
 48,
 108,
 48,
 112,
 198,
 175,
 48,
 119,
 198,
 176,
 35,
 71,
 88,
 80,
 88,
 35,
 108,
 48,
 110,
 120,
 48,
 115,
 198,
 176,
 48,
 76,
 200,
 163,
 87,
 68,
 85,
 48,
 112,
 100,
 35,
 100,
 48,
 200,
 164,
 198,
 188,
 117,
 48,
 108,
 48,
 112,
 198,
 175,
 48,
 119,
 198,
 176,
 35,
 71,
 88,
 80,
 88,
 35,
 104,
 48,
 119,
 100,
 48,
 100,
 35,
 100,
 48,
 112,
 120,
 117,
 48,
 100,
 48,
 200,
 164,
 198,
 188,
 117,
 35,
 100,
 48,
 200,
 164,
 198,
 188,
 117,
 48,
 71,
 88,
 229,
 133,
 132,
 229,
 133,
 131,
 35,
 108,
 48,
 111,
 198,
 164,
 48,
 198,
 164,
 200,
 164,
 48,
 117,
 100,
 48,
 113,
 108,
 48,
 112,
 100,
 35,
 100,
 48,
 113,
 100,
 35,
 200,
 164,
 100,
 48,
 111,
 108,
 112,
 48,
 100,
 48,
 200,
 164,
 198,
 188,
 117,
 48,
 112,
 100,
 35,
 52,
 35,
 74,
 198,
 157,
 35,
 68,
 81,
 49,
 81,
 68,
 35,
 52,
 51,
 35,
 43,

In [21]:
split = tokenized_dataset.train_test_split(
    test_size=0.1,
    seed=42,
    shuffle=True
)

In [22]:
train_dataset = split["train"]
val_dataset = split["test"]

In [23]:
def compute_metrics(eval_preds, tokenizer):
    """
    Oblicz Geometric Mean of BLEU and chrF++
    Dokładnie jak w evaluation konkursu
    """
    predictions, labels = eval_preds
    
    if isinstance(predictions, tuple):
        predictions = predictions[0]

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label. strip() for label in decoded_labels]
    
    bleu = sacrebleu.corpus_bleu(
        decoded_preds, 
        [decoded_labels],
        tokenize='13a'
    )

    chrf = sacrebleu.corpus_chrf(
        decoded_preds,
        [decoded_labels],
        word_order=2
    )
    
    geometric_mean = gmean([bleu.score, chrf.score])
    
    return {
        'bleu': bleu.score,
        'chrf': chrf.score,
        'geometric_mean': geometric_mean
    }

In [24]:
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    save_strategy='epoch',
    learning_rate=CONFIG['learning_rate'],
    per_device_train_batch_size=CONFIG['batch_size'],
    per_device_eval_batch_size=CONFIG['batch_size'],
    gradient_accumulation_steps=CONFIG['gradient_accumulation_steps'],
    weight_decay=CONFIG['weight_decay'],
    num_train_epochs=CONFIG['num_epochs'],
    warmup_steps=CONFIG['warmup_steps'],
    fp16=CONFIG['fp16'],
    # predict_with_generate=True,
    generation_max_length=CONFIG['max_target_length'],
    logging_dir='./logs',
    logging_steps=100,
    save_total_limit=3,
)

/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [25]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=lambda x: compute_metrics(x, tokenizer)
)

/tmp/ipykernel_17/4065446809.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [26]:
#trainer.train()


# Draft

In [27]:

print(f"Vocab size: {tokenizer.vocab_size}")
print(f"Model vocab size: {model.config.vocab_size}")

Vocab size: 256
Model vocab size: 384


In [28]:
source = "1 qa₂-am ṣa-ru-pa-am"
target = "1 qa of refined (silver)"

In [29]:
inputs = tokenizer(source, return_tensors='pt')
labels = tokenizer(target, return_tensors='pt')

In [30]:
inputs

{'input_ids': tensor([[ 52,  35, 116, 100, 229, 133, 133,  48, 100, 112,  35, 228, 188, 166,
         100,  48, 117, 120,  48, 115, 100,  48, 100, 112,   1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1]])}

In [31]:
model.eval()
with torch.no_grad():
    outputs = model(
        input_ids=inputs['input_ids'],
        labels=labels['input_ids']
    )

In [32]:
outputs.loss

tensor(5.4828)

In [33]:
outputs.logits.shape

torch.Size([1, 25, 384])